In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn.model_selection import validation_curve
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeClassifier

In [2]:
voices = pd.read_csv("voice.csv")

Gender_mapping = {'male':0, 'female':1}
voices = voices.replace({'label':Gender_mapping})

voices = voices.drop(["maxdom","mindom", "dfrange", "meandom", "mode", "maxfun", "minfun", "modindx", "skew","Q75","kurt"],axis = 1)
voices = voices.dropna()
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
features = ['meanfreq','sd','median','Q25','IQR','sp.ent','sfm','centroid','meanfun']
voices[features] = scaler.fit_transform(voices[features].to_numpy())

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rf.get_params())

Parameters currently in use:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


In [3]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}


In [4]:
X = voices.drop(['label'],axis=1)
y = voices['label']

# List of features for later use
feature_list = list(X.columns)
# Convert to numpy arrays
import numpy as np
X = np.array(X)
y = np.array(y)

# Training and Testing Sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 42, stratify = y)

In [10]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 5 fold cross validation, 
# search across 50 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [11]:
rf_random.best_params_

{'n_estimators': 1400,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 30,
 'bootstrap': True}

In [12]:
rf_random.best_score_

0.931900621394014

In [13]:
rf_random.best_estimator_

RandomForestRegressor(max_depth=30, max_features='sqrt', min_samples_split=5,
                      n_estimators=1400)

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    print("Test_Labels: %s"%test_labels)
    print("Mape: %s"%mape)

    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy
base_model = RandomForestRegressor(n_estimators = 10, random_state = 42)
base_model.fit(X_train, y_train)
base_accuracy = evaluate(base_model, X_test, y_test)

Test_Labels: [0 0 0 1 0 1 1 0 0 1 0 1 1 0 0 0 1 1 0 0 1 0 1 0 1 0 0 0 1 1 0 1 1 0 1 0 1
 0 1 1 0 1 0 0 0 0 1 1 0 1 0 0 1 0 1 1 0 1 1 0 1 1 0 1 0 0 0 1 0 1 1 0 0 0
 0 1 0 1 1 1 0 0 0 1 1 1 1 0 0 1 0 0 1 0 1 0 0 1 0 0 1 0 1 0 1 1 0 1 1 0 1
 1 1 1 1 0 1 1 1 0 0 1 1 1 0 0 1 1 0 0 1 0 1 0 0 1 1 0 0 0 1 1 0 1 1 1 0 1
 0 0 1 0 1 0 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 1 1 0 1 0 1 0 1 1 0 1 1 0 0 0 1
 1 1 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0 1 0 0 1 1 1 0 0 1 0 1 1 1
 0 0 0 1 1 0 1 0 0 0 1 1 0 0 1 0 1 0 0 1 1 1 1 1 1 1 0 0 0 1 0 1 0 0 0 1 1
 0 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 1 1 0 0 1 1 0 0 0 0 1 0 0 0 0 1 1 0
 0 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0 1 1 1 1 1 0 0 1 1 0 1 1 0 1 1 1 0 0 1 1 1
 0 1 1 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 1 0 1 1 1 0 1 0 1 1 1 1 1 0 0 1 0 0 1
 0 0 1 0 1 1 1 0 0 0 1 0 1 0 0 1 1 1 0 1 0 0 1 0 0 1 0 0 0 1 0 0 1 1 0 1 1
 1 0 1 1 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1 0 1 1 0 1 0 1 0
 1 1 0 0 1 1 1 1 1 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 1
 1 0 0 1 0 1

<ipython-input-47-ba6660942985>:4: RuntimeWarning: divide by zero encountered in true_divide
  mape = 100 * np.mean(errors / test_labels)
<ipython-input-47-ba6660942985>:4: RuntimeWarning: invalid value encountered in true_divide
  mape = 100 * np.mean(errors / test_labels)


In [ ]:
best_random = rf_random.best_estimator_
random_accuracy = evaluate(best_random, X_test, y_test)

Test_Labels: [0 0 0 1 0 1 1 0 0 1 0 1 1 0 0 0 1 1 0 0 1 0 1 0 1 0 0 0 1 1 0 1 1 0 1 0 1
 0 1 1 0 1 0 0 0 0 1 1 0 1 0 0 1 0 1 1 0 1 1 0 1 1 0 1 0 0 0 1 0 1 1 0 0 0
 0 1 0 1 1 1 0 0 0 1 1 1 1 0 0 1 0 0 1 0 1 0 0 1 0 0 1 0 1 0 1 1 0 1 1 0 1
 1 1 1 1 0 1 1 1 0 0 1 1 1 0 0 1 1 0 0 1 0 1 0 0 1 1 0 0 0 1 1 0 1 1 1 0 1
 0 0 1 0 1 0 1 1 1 0 0 0 1 0 0 0 1 0 1 1 0 1 1 0 1 0 1 0 1 1 0 1 1 0 0 0 1
 1 1 0 1 1 1 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0 1 0 0 1 1 1 0 0 1 0 1 1 1
 0 0 0 1 1 0 1 0 0 0 1 1 0 0 1 0 1 0 0 1 1 1 1 1 1 1 0 0 0 1 0 1 0 0 0 1 1
 0 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 1 1 0 0 1 1 0 0 0 0 1 0 0 0 0 1 1 0
 0 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0 1 1 1 1 1 0 0 1 1 0 1 1 0 1 1 1 0 0 1 1 1
 0 1 1 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 1 0 1 1 1 0 1 0 1 1 1 1 1 0 0 1 0 0 1
 0 0 1 0 1 1 1 0 0 0 1 0 1 0 0 1 1 1 0 1 0 0 1 0 0 1 0 0 0 1 0 0 1 1 0 1 1
 1 0 1 1 0 1 1 0 1 0 1 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 1 0 1 1 0 1 0 1 0
 1 1 0 0 1 1 1 1 1 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 1
 1 0 0 1 0 1

<ipython-input-47-ba6660942985>:4: RuntimeWarning: divide by zero encountered in true_divide
  mape = 100 * np.mean(errors / test_labels)
<ipython-input-47-ba6660942985>:4: RuntimeWarning: invalid value encountered in true_divide
  mape = 100 * np.mean(errors / test_labels)


In [ ]:
print('Improvement of {:0.2f}%.'.format( 100 * (random_accuracy - base_accuracy) / base_accuracy))

Improvement of nan%.
